# 1. Define Libraries

In [ ]:

# Core Python libraries
import os
import tempfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Data manipulation and visualization
from docx import Document
from docx.shared import Inches
import seaborn as sns

# Image processing

# Machine Learning models and preprocessing
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

# Machine Learning classifiers
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier

# External libraries
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import label_binarize
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tempfile
from docx import Document
from docx.shared import Inches
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, jaccard_score, matthews_corrcoef,
    cohen_kappa_score, roc_curve
)
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import os
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder


import pandas as pd
import numpy as np
import pandas as pd


from sklearn.ensemble import ExtraTreesClassifier

# External libraries
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
sns.set_theme(style='darkgrid', palette='pastel')
color = sns.color_palette(palette='pastel')

In [ ]:

def train_and_evaluate_export(
    classifiers: dict,
    train_features, train_labels,
    test_features, test_labels,
    history_dict: dict = None,
    word_file: str = "results.docx"
):
    # Prepare Word documenat
    doc =  Document()  #Document(word_file) if os.path.exists(word_file) else Document()

    # If labels are one-hot/multilabel, convert them to integer labels
    if isinstance(test_labels, np.ndarray) and test_labels.ndim > 1:
        test_true = np.argmax(test_labels, axis=1)
    else:
        test_true = test_labels
    

    unique_labels = np.unique(test_true)
    num_classes = len(unique_labels)
    accuracies = []
    tmp_files = []
    for name, model in classifiers.items():
        print(name)
        
        roc_data = []
        # Train
        fitted = model.fit(train_features, train_labels)
        # Predict
        y_pred_raw = model.predict(test_features)

        # If predict returns one-hot/multilabel, convert to class indices
        if isinstance(y_pred_raw, np.ndarray) and y_pred_raw.ndim > 1:
            y_pred = np.argmax(y_pred_raw, axis=1)
        else:
            y_pred = y_pred_raw
        
        # Try to get probabilities
        try:
            y_proba = model.predict_proba(test_features)
        except Exception:
            y_proba = None

        # Confusion matrix
        cm = confusion_matrix(test_true, y_pred)

        # Specificity per class
        specs = []
        for i in range(cm.shape[0]):
            tn = cm.sum() - (cm[i,:].sum() + cm[:,i].sum() - cm[i,i])
            fp = cm[:,i].sum() - cm[i,i]
            specs.append(tn / (tn + fp) if (tn + fp) > 0 else 0)
        avg_spec = np.mean(specs)

        # Compute metrics
        metrics = {
            "Accuracy": accuracy_score(test_true, y_pred),
            "Precision": precision_score(test_true, y_pred, average='weighted', zero_division=0),
            "Recall": recall_score(test_true, y_pred, average='weighted', zero_division=0),
            "F1 Score": f1_score(test_true, y_pred, average='weighted', zero_division=0),
            "Jaccard Index": jaccard_score(test_true, y_pred, average='weighted', zero_division=0),
            "Matthews CorrCoef": matthews_corrcoef(test_true, y_pred),
            "Cohen’s Kappa": cohen_kappa_score(test_true, y_pred),
            "Specificity": avg_spec,
        }

        # AUC
        auc = np.nan
        if y_proba is not None:
            try:
                if num_classes == 2:
                    # assume probabilities are shape (n_samples,2)
                    auc = roc_auc_score(test_true, y_proba[:,1])
                    fpr, tpr, _ = roc_curve(test_true, y_proba[:,1])
                    roc_data.append((name, fpr, tpr, auc))
                else:
                    # Multi-class: One-vs-Rest ROC for each class
                    y_bin = label_binarize(test_true, classes=unique_labels)
                    for i, cls in enumerate(unique_labels):
                        fpr, tpr, _ = roc_curve(y_bin[:,i], y_proba[:,i])
                        auc = roc_auc_score(y_bin[:,i], y_proba[:,i])
                        roc_data.append((f"Class {cls}", fpr, tpr, auc))
            except Exception:
                pass
        metrics["AUC"] = auc

        accuracies.append((name, metrics["Accuracy"] * 100))

        # Write to Word
        doc.add_heading(f"Evaluation: {name}", level=1)
        for m, v in metrics.items():
            doc.add_paragraph(f"{m}: {v if not np.isnan(v) else 'N/A'}")

        # Confusion matrix plot
        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmp:
            plt.figure(figsize=(4,4))
            sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
            plt.title(name)
            plt.xlabel("Predicted")
            plt.ylabel("True")
            plt.tight_layout()
            plt.savefig(tmp.name, dpi=300)
            plt.close()
            doc.add_picture(tmp.name, width=Inches(4))
            tmp_files.append(tmp.name)  # store path for later deletion

        # Loss plot if available
        if history_dict and name in history_dict:
            hist = history_dict[name].history
            with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmp:
                plt.figure(figsize=(6,6))
                plt.plot(hist['loss'], label='Train Loss')
                if 'val_loss' in hist:
                    plt.plot(hist['val_loss'], label='Val Loss')
                plt.title(f"Loss: {name}")
                plt.xlabel('Epoch')
                plt.ylabel('Loss')
                plt.legend()
                plt.tight_layout()
                plt.savefig(tmp.name, dpi=300)
                plt.close()
                doc.add_picture(tmp.name, width=Inches(6))
                tmp_files.append(tmp.name)  # store path for later deletion

        # Combined ROC for binary classifiers
        if roc_data:
            doc.add_heading("Combined ROC Curves", level=1)
            with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmp:
                plt.figure(figsize=(6,6))
                for label, fpr, tpr, auc_score in roc_data:
                    plt.plot(fpr, tpr, label=f"{label} (AUC={auc_score:.2f})")
                plt.plot([0,1],[0,1], linestyle='--', color='black')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('ROC Curves')
                plt.legend(fontsize=8)
                plt.tight_layout()
                plt.savefig(tmp.name, dpi=300)
                plt.close()
                doc.add_picture(tmp.name, width=Inches(6))
                tmp_files.append(tmp.name)  # store path for later deletion
        print(metrics["Accuracy"])
    doc.save(word_file)
    # Delete temp files safely
    for f in tmp_files:
        os.unlink(f)
    print(f"✅ Results saved to {word_file}")
    return accuracies

In [ ]:
CSV_PATH = "../aug_manifest_filled.csv"          # <- change
COL_ORIGINAL = "origin"      # <- change if needed
COL_LABEL = "disease_name"     # <- change if needed
SEED = 42
N_SPLITS = 5

df = pd.read_csv(CSV_PATH).copy()

# Keep only unique originals (drop duplicate rows by original)
df_unique = df.drop_duplicates(subset=[COL_ORIGINAL], keep="first").reset_index(drop=True)

# 5-fold on unique originals, stratified by disease_name
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
df_unique["fold"] = -1

X_dummy = df_unique[[COL_ORIGINAL]]   # placeholder
y = df_unique[COL_LABEL]

for fold_id, (_, val_idx) in enumerate(skf.split(X_dummy, y), start=1):
    df_unique.loc[val_idx, "fold"] = fold_id

# Check: this should be ~7909 per fold if total unique originals ~39545
print("Unique originals:", len(df_unique))
print(df_unique["fold"].value_counts().sort_index())



et_base = ExtraTreesClassifier(
    n_estimators=800,          # more trees for stability
    max_depth=None,           # let trees grow full
    min_samples_leaf=1,
    max_features="sqrt",      # random subspace often helps with lots of features
    random_state=42
)
rf_base = RandomForestClassifier(
    n_estimators=800,          # more trees for stability
    max_depth=None,           # let trees grow full
    min_samples_leaf=1,
    max_features="sqrt",      # random subspace often helps with lots of features
    random_state=42
)
OvR_classifiers = {
    "ET-OvR": OneVsRestClassifier(et_base, n_jobs=-1),
    "RF-OvR": OneVsRestClassifier(rf_base, n_jobs=-1)
}


classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Extra Trees": ExtraTreesClassifier(n_estimators=100, random_state=42,class_weight="balanced"),
    "Bagging Classifier": BaggingClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "LightGBM": LGBMClassifier(random_state=42)
}


In [ ]:

# -----------------------------
# Load CSV files
# -----------------------------
folds_df = pd.read_csv("../folds.csv")
aug_df = pd.read_csv("../aug_manifest_filled.csv")
features_df = pd.read_csv("../glcm_features_fold1.csv") 

features_df["path"] = "BreaKHis_v1/" + features_df["path"].astype(str).str.replace("\\", "/", regex=False).str.split("BreaKHis_v1/").str[-1]


# -----------------------------
# Select fold
# -----------------------------

for SELECTED_FOLD in [1,2,3,4,5]:
    test_files  = df_unique.loc[df_unique["fold"] == SELECTED_FOLD, "origin"].unique()
    train_files = df_unique.loc[df_unique["fold"] != SELECTED_FOLD, "origin"].unique()
    train_meta = aug_df[
        aug_df["origin"].isin(train_files)
    ].copy()

    # -----------------------------
    # Testing set (clean samples only)
    # -----------------------------
    test_meta = aug_df[
        (aug_df["origin"].isin(test_files)) &
        (aug_df["crop"] == "center") &
        (aug_df["flip"] == "none") &
        (aug_df["rotation"] == 0)
    ].copy()

    # -----------------------------
    # Merge with features
    # -----------------------------
    train_df = aug_df.merge(
        features_df,
        left_on="filename",
        right_on="path",
        how="inner"
    )

    test_df = test_meta.merge(
        features_df,
        left_on="filename",
        right_on="path",
        how="inner"
    )


    # -----------------------------
    # Define labels
    # -----------------------------
    y_train = train_df["disease_name"]
    y_test = test_df["disease_name"]

    # -----------------------------
    # Drop non-feature columns
    # -----------------------------
    NON_FEATURE_COLS = {
        "filename", "path", "origin",
        "crop", "flip", "rotation",
        "disease_name", "disease_type",
        "mag", "fold"
    }

    X_train = train_df.drop(columns=[c for c in NON_FEATURE_COLS if c in train_df.columns])
    X_test = test_df.drop(columns=[c for c in NON_FEATURE_COLS if c in test_df.columns])
    X_train_path = train_df["path"]
    X_test_path = test_df["path"]
    # -----------------------------
    # Final sanity checks
    # -----------------------------
    assert len(X_train) == len(y_train)
    assert len(X_test) == len(y_test)

    print("Train shape:", X_train.shape, "Labels:", y_train.nunique())
    print("Test shape:", X_test.shape)



    # =========================================
    # 4) STRATIFIED 50:50 SPLIT FOR TRAIN AND TEST
    # =========================================

    sss_train = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
    sss_test  = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)

    # ---- Stratified 50:50 split for TRAIN ----
    for idx1, idx2 in sss_train.split(X_train, y_train):
        train_features  = X_train.iloc[idx1]
        train_labels    = y_train.iloc[idx1]
        train_path      = X_train_path.iloc[idx1]

        train2_features = X_train.iloc[idx2]
        train2_labels   = y_train.iloc[idx2]
        train_path2      = X_train_path.iloc[idx2]

    # ---- Stratified 50:50 split for TEST ----
    for idx1, idx2 in sss_test.split(X_test, y_test):
        test_features   = X_test.iloc[idx1]
        test_labels     = y_test.iloc[idx1]
        test_path       = X_test_path[idx1]

        test2_features  = X_test.iloc[idx2]
        test2_labels    = y_test.iloc[idx2]
        test_path2      = X_test_path[idx2]


    print("train_features:", train_features.shape, "train2_features:", train2_features.shape)
    print("test_features:",  test_features.shape,  "test2_features:",  test2_features.shape)


    # =========================================
    # 5) ENCODE LABELS INTO NUMERICAL FORM (FOR ML MODELS)
    # =========================================

    # Fit encoder on ALL labels from the selected fold to keep mapping consistent
    all_labels = pd.concat([y_train, y_test]).astype(str).values

    le = LabelEncoder()
    le.fit(all_labels)

    # Encode each split
    train_labels_enc  = le.transform(train_labels)
    train2_labels_enc = le.transform(train2_labels)

    test_labels_enc   = le.transform(test_labels)
    test2_labels_enc  = le.transform(test2_labels)

    # Optional: print label→index mapping
    label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print("Label mapping:")
    for k, v in label_mapping.items():
        print(f"  {k} -> {v}")

    accuracies = train_and_evaluate_export(
        OvR_classifiers,
        train_features, train_labels_enc,
        test_features, test_labels_enc,
        word_file="OvR results Multiclass fold-"+ str(SELECTED_FOLD)+".docx"
    )
    print(accuracies)


    
    # 1) Get class-probability predictions for train2 and test2
    et_ovr = OvR_classifiers["ET-OvR"]

    ET_train_proba = et_ovr.predict_proba(train2_features)  # shape (n_train2, n_classes)
    ET_test_proba  = et_ovr.predict_proba(test2_features)   # shape (n_test2,  n_classes)

    # Optional sanity check
    print("ET_train_proba shape:", ET_train_proba.shape)
    print("ET_test_proba shape:",  ET_test_proba.shape)

    # 2) If you only need numeric arrays for the meta-model:
    #    Simply stack original features and probabilities horizontally.

    new_ET_OvR_training_features = np.hstack([train2_features, ET_train_proba])
    new_ET_OvR_test_features     = np.hstack([test2_features,  ET_test_proba])

    print("new_ET_OvR_training_features shape:", new_ET_OvR_training_features.shape)
    print("new_ET_OvR_test_features shape:",     new_ET_OvR_test_features.shape)

    # 3) Train/evaluate second-stage classifiers using stacked features

    accuracies = train_and_evaluate_export(
        classifiers,
        new_ET_OvR_training_features, train2_labels_enc,
        new_ET_OvR_test_features,     test2_labels_enc,
        word_file="ET-OvR as feature extractor Multiclass results fold-"+ str(SELECTED_FOLD)+".docx",
    )

    print("Stacked-model accuracies:")
    print(accuracies)
    # 1) Get class-probability predictions for train2 and test2
    RF_ovr = OvR_classifiers["RF-OvR"]

    RF_train_proba = RF_ovr.predict_proba(train2_features)  # shape (n_train2, n_classes)
    RF_test_proba  = RF_ovr.predict_proba(test2_features)   # shape (n_test2,  n_classes)

    # Optional sanity check
    print("RF_train_proba shape:", RF_train_proba.shape)
    print("RF_test_proba shape:",  RF_test_proba.shape)

    # 2) If you only need numeric arrays for the meta-model:
    #    Simply stack original features and probabilities horizontally.

    new_RF_OvR_training_features = np.hstack([train2_features, RF_train_proba])
    new_RF_OvR_test_features     = np.hstack([test2_features,  RF_test_proba])

    print("new_RF_OvR_training_features shape:", new_RF_OvR_training_features.shape)
    print("new_RF_OvR_test_features shape:",     new_RF_OvR_test_features.shape)

    # 3) Train/evaluate second-stage classifiers using stacked features

    accuracies = train_and_evaluate_export(
        classifiers,
        new_RF_OvR_training_features, train2_labels_enc,
        new_RF_OvR_test_features,     test2_labels_enc,
        word_file="RF-OvR as feature extractor Multiclass results fold-"+ str(SELECTED_FOLD)+".docx",
    )

    print("Stacked-model accuracies:")
    print(accuracies)


In [ ]:
def generate_report_from_test20_v2(test20_df, ovr_classifier, final_classifier, class_names,
                                   save_path="random_report.png", dpi=300):
    """
    Generate A5 visualization for a random sample from test20_df.
    Page split into 8 slots:
      1- Final decision + probability %
      2- Histology image
      3- OvR probabilities
      4–8- GLCM feature groups
    """
    import warnings
    warnings.filterwarnings("ignore", message="X does not have valid feature names")
    plt.rcParams.update({'font.size': 7})
    plt.rcParams.update({
        "axes.facecolor": "white",   # background of axes
        "figure.facecolor": "white"  # background of entire figure
    })
    plt.rcParams.update({
        "axes.grid": True,          # turn on grid
        "grid.color": "gray",       # gridline color
        "grid.linestyle": "--",     # dashed
        "grid.linewidth": 0.5       # thin
    })
    # --- Pick random row ---
    idx = np.random.choice(test20_df.index)
    row = test20_df.loc[idx]

    # --- Extract raw GLCM features ---
    glcm_cols = [c for c in test20_df.columns
                 if any(k in c for k in ["contrast", "energy", "homogeneity", "homogynity", "correlation", "entropy"])]
    glcm_features = row[glcm_cols].values.reshape(1, -1)

    # --- Compute OvR probabilities ---
    proba_array = ovr_classifier.predict_proba(glcm_features)  # shape (1, n_classes)
    class_labels = ovr_classifier.classes_
    proba_cols = [f"ET_OvR_proba_{cls}" for cls in class_labels]
    proba_df = pd.DataFrame(proba_array, index=[idx], columns=proba_cols)

    # --- Concatenate to form final input ---
    new_features = pd.concat([pd.DataFrame(glcm_features, index=[idx], columns=glcm_cols),
                              proba_df], axis=1)

    # --- Predict with final classifier ---
    final_proba = final_classifier.predict_proba(new_features)[0]
    pred_idx = int(np.argmax(final_proba))
    pred_name = class_names[pred_idx]
    true_label = row["TumorSubtype"]

    # --- Visualization Layout: 4 rows × 2 columns = 8 slots ---
    fig, axes = plt.subplots(4, 2, figsize=(8.3, 11.6))  # A4 size; change to (5.8, 8.3) for A5
    axes = axes.flatten()

    # Slot 1: Final decision text
    axes[0].axis("off")
    axes[0].text(0.5, 0.5,
                 f"Final Prediction: {pred_name}\n"
                 f"True Label: {true_label}\n"
                 f"Confidence: {final_proba[pred_idx]*100:.2f}%",
                 fontsize=14, ha="center", va="center", weight="bold")

    # Slot 2: Histology image
    img_path = row["path"].replace("BreaKHis_v1/", "E:/BreaKHis_augmentedv2/BreaKHis_v1/")
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        axes[1].imshow(img)
    else:
        axes[1].imshow(np.random.rand(100, 100, 3))
    axes[1].axis("off")
    axes[1].set_title("Sample Image")

    # Slot 3: OvR probabilities
    y_pos = np.arange(len(class_names))
    axes[2].barh(y_pos, proba_array[0], color="skyblue")
    axes[2].barh(pred_idx, proba_array[0][pred_idx], color="orange")
    axes[2].set_yticks(y_pos, class_names, fontsize=7)
    axes[2].set_xlim(0, 1)
    axes[2].set_xlabel("Probability")
    axes[2].set_title("OvR Probabilities")    # Slots 4–8: GLCM groups (mean ± std across distances/angles per band)
    bands = ["cA", "cH1", "cV1", "cD1", "cH2", "cV2", "cD2"]
    metrics = [
        ("Contrast", "contrast"),
        ("Energy", "energy"),
        ("Homogeneity", "homogeneity"),
        ("Correlation", "correlation"),
        ("Entropy", "entropy"),
    ]

    def band_metric_cols(band: str, metric_key: str):
        """Collect all columns for a band+metric.
        Supports both:
          - {band}_{metric}_d{d}_a{a} (e.g., cA_contrast_d1_a0)
          - {band}_entropy (no distance/angle)
        """
        if metric_key == "entropy":
            return [c for c in glcm_cols if c == f"{band}_entropy" or c.startswith(f"{band}_entropy_")]
        # typical case: cA_contrast_d1_a0, ...
        prefix = f"{band}_{metric_key}_"
        return [c for c in glcm_cols if c.startswith(prefix)]

    for ax, (title, key) in zip(axes[3:], metrics):
        means, stds = [], []
        for band in bands:
            cols = band_metric_cols(band, key)
            if len(cols) == 0:
                means.append(0.0)
                stds.append(0.0)
                continue
            vals = pd.to_numeric(pd.Series([row[c] for c in cols]), errors="coerce").dropna().values
            if vals.size == 0:
                means.append(0.0)
                stds.append(0.0)
                continue
            means.append(float(np.mean(vals)))
            stds.append(float(np.std(vals, ddof=1)) if vals.size > 1 else 0.0)

        ax.barh(range(len(bands)), means, xerr=stds, color="steelblue",
                ecolor="black", capsize=3)
        ax.set_yticks(range(len(bands)), bands, fontsize=8)
        ax.set_title(title)

    # Remove any unused slots (if fewer than 8 GLCM groups)
    for ax in axes[3+len(metrics):]:
        ax.axis("off")
    for ax in fig.axes:
        for spine in ax.spines.values():
            spine.set_edgecolor("gray")
            spine.set_linewidth(0.8)
    plt.tight_layout()
    plt.savefig(save_path, dpi=dpi, bbox_inches="tight")
    plt.show()

    return idx, pred_name, true_label


In [ ]:
# Assumptions:
# - You already have: test2_features (pd.DataFrame), test2_labels (pd.Series or array-like)
# - You already have trained: ovr_classifier, final_classifier
# - You already have the function: generate_report_from_test20_v2(...)
# - Optional (recommended): test2_filepaths (pd.Series indexed like test2_features) with real image paths

import numpy as np
import pandas as pd
import cv2

# -----------------------------
# 1) Build test20_df as expected by the report function
#    - Must contain: GLCM feature columns, "filepaths", "TumorSubtype"
# -----------------------------
test20_df = test2_features.copy()

# Ensure GLCM columns ordering matches OvR training (prevents silent column-order bugs)
if hasattr(RF_ovr, "feature_names_in_"):
    ovr_cols = list(RF_ovr.feature_names_in_)
    missing = [c for c in ovr_cols if c not in test20_df.columns]
    if missing:
        raise ValueError(f"Missing OvR feature columns in test2_features: {missing[:10]} ... ({len(missing)} total)")
    # Put OvR features first, keep any extra columns after (won't hurt)
    extra = [c for c in test20_df.columns if c not in ovr_cols]
    test20_df = test20_df[ovr_cols + extra]

# Add true label column (string form is fine)
test20_df["TumorSubtype"] = pd.Series(test2_labels, index=test20_df.index).astype(str)
test2_filepaths = test_path2
# Add filepaths (use your real file path series if you have it)
if "filepaths" not in test20_df.columns:
    if "test2_filepaths" in globals() and isinstance(test2_filepaths, (pd.Series, pd.Index)):
        test20_df["path"] = pd.Series(test2_filepaths, index=test20_df.index).astype(str)
        print(test20_df["path"].head())
    else:
        # Fallback (report will show random image if cv2.imread fails)
        test20_df["path"] = ""
test20_df["path"] = pd.Series(test_path2, index=test20_df.index).astype(str)
print(test20_df.head())
# -----------------------------
# 2) class_names for display
#    Prefer your label encoder classes if you have them; otherwise derive from test2_labels
# -----------------------------
if "class_names" not in globals() or class_names is None or len(class_names) == 0:
    # Safe default: sorted unique label strings from this fold
    class_names = sorted(test20_df["TumorSubtype"].unique().tolist())

# -----------------------------
# 3) Generate N random reports from test2_features
# -----------------------------
N_REPORTS = 50
for i in range(N_REPORTS):
    save_path = f"report_test2_{i+1:02d}.png"
    idx, pred_name, true_label = generate_report_from_test20_v2(
        test20_df=test20_df,
        ovr_classifier=RF_ovr,
        final_classifier=classifiers['Extra Trees'],
        class_names=class_names,
        save_path=save_path,
        dpi=300,
    )
    print(f"Saved {save_path} | idx={idx} | pred={pred_name} | true={true_label}")


In [ ]:

import os
import zipfile
zip_name = "reports2.zip"
with zipfile.ZipFile(zip_name, "w") as zipf:
    for i in range(20):
        filename = os.path.join("/kaggle/working", f"{i}.png")
        if os.path.exists(filename):
            zipf.write(filename, os.path.basename(filename))
        else:
            print(f"⚠️ Missing file: {filename}")

print(f"✅ Created {zip_name}")